## Lesson 1

## Poker Program

### Problem understanding of a poker game

## Steps:

- Understand the problem
- Specification
- Design the code

## Notions:
- poker(hands) `->` hand (Return a list of best hands allowing ties)
- Hand consists of 5 cards, each card has rank (2,3,5,...) and suit (Ace, heart, Spade, clove)
- Hand mapped to a rank, i.e, n-kind (e.g 2 pairs), straigh, flush


In [2]:
import random
import itertools


allranks = '23456789TJQKA'
redcards = [r+s for r in allranks for s in 'DH']
blackcards = [r+s for r in allranks for s in'SC']

def best_wild_hand(hand):
    """
    Try all values for jokers in all 5-card selections
    """
    hands = set(best_hand(h) for h in itertools.product(*map(replacements, hand)))
    return max(hands, key = hand_rank)

def replacements(card):
    """
    Return a list of the possible replacement for a card
    """
    if card == '?B': return blackcards
    elif card == '?R': return redcards
    else: return [card]
    
    
def best_hand(hand):
    """
    Return the best 5 card hand from a 7-card hand
    """
    return max(itertools.combinations(hand, 5), key = hand_rank)
    

def deal(numhands, n = 5, deck = [r + s for r in '23456789TJQKA' for s in 'SHDC']):
    """
    shuffle the deck and return a list of hands with n-card
    """
    
    random.shuffle(deck)
    return [deck[n*i:n*(i+1)] for i in xrange(numhands)]

def poker(hands):
    """
    Return the best hand
    poker[hand, hand, ...] => [hand, ...]
    """
    #return max([(hand, handRank(hand)) for hand in hands], key = lambda item:item[1])[0]
    return allmax(hands, key = hand_rank)

def allmax(iterable, key = None):
    """
    Return a list of all items equal to the max of the iterable
    This function is to allow max values with ties
    """
    
    result, maxval = [], None
    key = key or (lambda x: x)
    for x in iterable:
        xval = key(x)
        if not result or xval > maxval:
            result, maxval = [x], xval
        elif xval == maxval:
            result.append(x)
    return result

count_rankings = {(5, ): 10, (4, 1): 7, (3, 2): 6, (3, 1, 1): 3, 
                  (2, 2, 1): 2, (2, 1, 1, 1): 1, (1, 1, 1, 1, 1): 0}

def hand_rank(hand):
    """
    Return a value indicating the ranking of a hand
    There are 9 ranks (a value of 0-8), the highest is straight flush (sf)
    count is the count of each rank; ranks list corresponding ranks
    e.g '7 T 7 9 7' => counts = (3, 1, 1); ranks = (7, 10, 9)
    """
    groups = group(['--23456789TJQKA'.index(r) for r,s in hand])
    counts, ranks = unzip(groups)
    if ranks == (14, 5, 4, 3, 2):
        ranks = (5, 4, 3, 2, 1)
    straight = len(ranks) == 5 and max(ranks) - min(ranks) == 4
    flush = len(set(s for r,s in hand)) == 1
    return max(count_rankings[counts], 4 * straight + 5 * flush), ranks


def group(items):
    """
    Return a list of [(count, x)], the highest count first, then highest x first
    """
    groups = [(items.count(x), x) for x in set(items)]
    return sorted(groups, reverse = True)

def unzip(pairs): 
    """
    Return [(3, 7), (1, 10), (1, 9)] => [(3, 1, 1), (7, 10, 9)]
    """
    return zip(*pairs)
    
def test():
    """
    Test cases for the functions in poker program
    """
    
    sf = "6C 7C 8C 9C TC".split() # straight flush
    fk = "9D 9H 9S 9C 7D".split() # four of a kind
    fh = "TD TC TH 7C 7D".split() # full house
    tp = "5S 5D 9H 9C 6S".split() # two pair
    s1 = "AS 2S 3S 4S 5C".split() # A-5 straight
    s2 = "2C 3C 4C 5S 6S".split() # 2-6 straight
    ah = "AS 2S 3S 4S 6C".split() # A high
    sh = "2S 3S 4S 6C 7D".split() # 7 high
    
    #fkranks = card_ranks(fk)
    #tpranks = card_ranks(tp)
    #assert card_ranks(sf) == [10, 9, 8, 7, 6]
    #assert card_ranks(fk) == [9, 9, 9, 9 ,7]
    #assert card_ranks(fh) == [10, 10, 10, 7, 7]
    #assert card_ranks(['AC', '3D', '4S', 'KH']) == [14, 13, 4, 3]
    #assert kind(4, fkranks) == 9
    #assert kind(3, fkranks) == None
    #assert kind(2, fkranks) == None
    #assert kind(1, fkranks) == 7
    #assert two_pair(fkranks) == None
    #assert two_pair(tpranks) == (9,5)
    #assert straight([9, 8, 7, 6, 5]) == True
    #assert straight([9, 8, 8, 6, 5]) == False
    #assert flush(sf) == True
    #assert flush(fk) == False
    #print(sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))) 
    
    assert poker([sf] + 99*[fh]) == [sf]
    assert poker([sf, fk]) == [sf]
    assert poker([sf, sf]) == [sf, sf]
    assert (sorted(best_hand("6C 7C 8C 9C TC 5C JS".split()))
            == ['6C', '7C', '8C', '9C', 'TC'])
    assert (sorted(best_hand("TD TC TH 7C 7D 8C 8S".split()))
            == ['8C', '8S', 'TC', 'TD', 'TH'])
    assert (sorted(best_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    assert (sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    assert (sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return "tests pass"
    
print test()

tests pass


## Hand rank frequency 

* source: wikipedia

- Straight flush: 0.0015%
-         4 kind: 0.024%
-    Full house: 0.140%
-            Flush: 0.196%
-         Straight: 0.39%
-           3 kind: 2.11%
-            2 Pair: 4.75%
-                Pair: 42.25%
-       High card: 50.11%


Note to estimate the hand rank frequency with need to sample at least

- 10/least common rank  ,i.e ~ 10/0.0015% = ~ 666666 or 700000


In [3]:
from collections import defaultdict

hand_names = {0: "High card", 
              1: "Pair",
              2: "2 Pair",
              3: "3 Kind",
              4: "Straight",
              5: "Flush",
              6: "Full house",
              7: "4 kind",
              9: "Straight Flush"}

def hand_percentages(n = 700 * 1000, hand_names = hand_names):
    
    """
    Sample n random hands and print a table of percentages for each type of hand
    """
    
    counts = defaultdict(int)
    for i in xrange(n/10):
        for hand in deal(10):
            ranking = hand_rank(hand)[0]
            counts[ranking] += 1
    for i in reversed(counts.keys()):
        print "%14s: %6.4f %%" %(hand_names[i], 100. * counts[i]/n)
        
hand_percentages(700000)

Straight Flush: 0.0019 %
        4 kind: 0.0237 %
    Full house: 0.1507 %
         Flush: 0.1967 %
      Straight: 0.3983 %
        3 Kind: 2.1199 %
        2 Pair: 4.7801 %
          Pair: 42.2644 %
     High card: 50.0643 %


## Shuffle algorithm

In [13]:
from collections import defaultdict
def shuffle_brute(deck):
    N = len(deck)
    swapped = [False] * N
    while not all(swapped):
        i, j = random.randrange(N), random.randrange(N)
        swapped[i] = swapped[j] = True
        swap(deck , i, j)
        
def shuffle_knuth(deck):
    N = len(deck)
    for i in xrange(N-1):
        swap(deck, i, random.randrange(i, N))
   
def swap(deck, i, j):
    #print 'swap', i, j
    deck[i], deck[j] = deck[j], deck[i]


def test_shuffler(shuffler, deck = 'abcd', n = 10000):
    counts = defaultdict(int)
    for i in xrange(n):
        input = list(deck)
        shuffler(input)
        counts[''.join(input)] += 1
    
    e = n * 1./factorial(len(deck))
    ok = all((0.9 <= counts[item]/e <= 1.1 for item in counts))
    name = shuffler.__name__
    print '%s(%s) %s' % (name, deck, ('ok' if ok else '***BAD***'))
    print '\n',
    for item, count in sorted(counts.items()):
        print "%s:%4.1f" % (item, count*100./n)
    print 
    
def test_shufflers(shufflers=[shuffle_brute, shuffle_knuth], decks = ['abc', 'ab']):
    for deck in decks:
        print 
        for f in shufflers:
            test_shuffler(f, deck)
            
def factorial(n): return 1 if (n <= 1) else n*factorial(n-1)

#print shuffle_knuth(list('abcd'))
test_shufflers()


shuffle_brute(abc) ***BAD***

abc: 4.7
acb:13.5
bac:14.1
bca:26.6
cab:27.1
cba:14.1

shuffle_knuth(abc) ok

abc:16.7
acb:16.4
bac:16.8
bca:16.4
cab:17.1
cba:16.6


shuffle_brute(ab) ***BAD***

ab:17.2
ba:82.8

shuffle_knuth(ab) ok

ab:49.9
ba:50.1



## Computing vs Doing

Note that shuffle (above function) returns None but it modifies directly the input

- Computing: return the result but it does not modify the input (e.g sqrt, sin, sorted) `->` a function
- Doing: like subroutines e.g shuffle, sort method

To test the computing is easier with `assert`:

- `sorted` as computing: `assert sorted([3,1,2]) == [1,2,3]`
- `.sort` as doing: 

    `input = [3,2,1]` # set the state
    
    `input.sort()`
    
    `input == [1,2,3]`

## Programming Principles

- Correct `->`  try to make a test case
- Efficiency `->` timing the running program
- Feature `->` allowing some flexibility